# 循环神经网络

## 1. 数据准备

将训练语料转换为字典形式

In [1]:
%matplotlib inline
import zipfile
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def load_data_jay_lyrics(num_sample=10000):
    """
    加载周杰伦歌词数据集
    """
    # 读取数据集
    with open('../dataset/jaychou_lyrics.txt') as f:
        corpus_chars = f.read()
    # 把换行符替换为空格
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[:num_sample]
    # 建立字符索引
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    # 将训练集中每个字符转换为索引
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    # 返回索引后的前num_sample个字符的文本，字符到索引的映射，索引到字符的映射，字符表大小
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

In [5]:
corpus_indices, char_to_idx, idx_to_char, vocab_size = load_data_jay_lyrics()

In [6]:
vocab_size

1027

In [7]:
len(corpus_indices)   # 10000

10000

In [8]:
char_to_idx['风']

11

In [9]:
idx_to_char[371]

'膀'

## 2. 采样

### 2.1 随机采样

下⾯的代码每次从数据⾥随机采样⼀个小批量。其中批量⼤小batch_size指每个小批量的样本数，num_steps为每个样本所包含的时间步数。在随机采样中，每个样本是原始序列上任意截取的⼀段序列。相邻的两个随机小批量在原始序列上的位置不⼀定相毗邻。因此，我们⽆法⽤⼀个小批量最终时间步的隐藏状态来初始化下⼀个小批量的隐藏状态。在训练模型时，每次随机采样前都需要重新初始化隐藏状态。

In [10]:
def data_iter_random(corpus_indices, batch_size, num_steps):
    '''
    corpus_indices: 词典按先后次序的索引
    batch_size: 每个批次的样本容量
    num_steps: 每个样本的长度
    '''
    num_examples = (len(corpus_indices) - 1) // num_steps  # 可取的样本数量
    epoch_size = num_examples // batch_size  # 总词汇数量 / (样本长度 * 样本数量)
    example_indices = list(range(num_examples))
    np.random.shuffle(example_indices)  # 打乱索引的顺序，即随机采样
      
    for i in range(epoch_size):
        # 每次读取batch_size个随机样本
        batch_indices = example_indices[i*batch_size: (i + 1)*batch_size ]
        X = [corpus_indices[j*num_steps: (j + 1)*num_steps] for j in batch_indices]
        Y = [corpus_indices[j*num_steps + 1: (j + 1)*num_steps + 1] for j in batch_indices]
        yield np.array(X), np.array(Y)

In [29]:
my_seq = list(range(30))
i = 0
for X, Y in data_iter_random(my_seq, batch_size=3, num_steps=6):
    print(i)
    print('X:\n', X, '\nY:\n', Y)
    i += 1

0
X:
 [[18 19 20 21 22 23]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]] 
Y:
 [[19 20 21 22 23 24]
 [ 7  8  9 10 11 12]
 [13 14 15 16 17 18]]


### 2.2 相邻采样

除对原始序列做随机采样之外，我们还可以令相邻的两个随机小批量在原始序列上的位置相毗邻。这时候，我们就可以用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态，从而使下一个小批量的输出也取决于当前小批量的输入，并如此循环下去。这对实现循环神经网络造成了两方面影响：一方面， 在训练模型时，我们只需在每一个迭代周期开始时初始化隐藏状态；另一方面，当多个相邻小批量通过传递隐藏状态串联起来时，模型参数的梯度计算将依赖所有串联起来的小批量序列。同一迭代周期中，随着迭代次数的增加，梯度的计算开销会越来越大。 为了使模型参数的梯度计算只依赖一次迭代读取的小批量序列，我们可以在每次读取小批量前将隐藏状态从计算图中分离出来。

In [11]:
# 本函数已保存在d2lzh包中方便以后使用
def data_iter_consecutive(corpus_indices, batch_size, num_steps):
    '''
    corpus_indices: 词典按先后次序的索引
    batch_size: 每个批次的样本容量
    num_steps: 每个样本的长度
    '''
    corpus_indices = np.array(corpus_indices)
    data_len = len(corpus_indices)  # 单词个数
    batch_len = data_len // batch_size  # 小批量的数量
    indices = corpus_indices[0: batch_size*batch_len].reshape(batch_size, batch_len)  # 先取总量，再塑形
    epoch_size = (batch_len - 1) // num_steps  # 批量数量
    for i in range(epoch_size):
        X = indices[:, i * num_steps: (i + 1) * num_steps]
        Y = indices[:, i * num_steps + 1: (i + 1) * num_steps + 1]
        yield X, Y

In [30]:
my_seq = list(range(30))
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print('X:\n', X, '\nY:\n', Y)

X:
 [[ 0  1  2  3  4  5]
 [15 16 17 18 19 20]] 
Y:
 [[ 1  2  3  4  5  6]
 [16 17 18 19 20 21]]
X:
 [[ 6  7  8  9 10 11]
 [21 22 23 24 25 26]] 
Y:
 [[ 7  8  9 10 11 12]
 [22 23 24 25 26 27]]


## 2. `one_hot`编码

对语料中的每个不同单词进行`one_hot`编码

In [46]:
def one_hot(array, vocab_size):
    '''
    array: 需要编码的索引
    vocab_size: 词典大小
    '''
    array = np.array(array)
    shape = list(array.shape) + [vocab_size]
    res = np.zeros(shape=shape)
    if len(shape) == 2:
        res[range(shape[0]), array] = 1
    elif len(shape) == 3:
        for i in range(shape[0]):
            for j in range(shape[1]):
                res[i, j, array[i, j]] = 1
    else:
        print('X超过2维!')

    return res

In [47]:
one_hot([[1, 2],[2, 3],[3, 4]], 10)

array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]]])

每次采样的小批量的形状是(批量大小, 时间步数)，以下函数将字符下标转换成字符的`one-hot`编码。

In [48]:
def to_onehot(X, size):
    """
    X: n*d, n为批量大小，d为输入个数
    """
    return [one_hot(x, size) for x in X]

In [49]:
X = np.arange(10).reshape((2, 5))
inputs = to_onehot(X, 10)

In [51]:
len(inputs), inputs[0].shape

(2, (5, 10))

初始化模型参数

In [ ]:
def inti_rnn_hidden